In [28]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

In [43]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Url of Mars Planet Science Website
url = "https://redplanetscience.com/"
browser.visit(url)

# Set up MongoDB
db = client.marsDB
collection = db.articles

#Clear and set time
db.articles.delete_many({})
time.sleep(1)

In [4]:
# request the set url
time.sleep(1)
html = browser.html

In [5]:
# parse and look at the html file
time.sleep(1)
soup=bs(html, 'html.parser')
print(soup.prettify)

<bound method Tag.prettify of <html><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
<link href="css/font.css" rel="stylesheet" type="text/css"/>
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
<title>News - Mars Exploration Program</title>
</head>
<body>
<div class="col-md-12">
<div class="row">
<nav class="navbar navbar-expand-lg navbar-light fixed-top">
<div class="container-fluid">
<a class="navbar-brand" href="#">
<img src="image/nasa.png" width="80"/><span class="logo">MARS Planet Science</span>
<span class="l

In [6]:
results = soup.find_all('div', class_='list_text')
#loop
for r in results:
    #pull titles
    title = r.find('div', class_='content_title').text
    
    #pull articles
    article = r.find('div', class_='article_teaser_body').text
    
    productdict = {
        'title': title,
        'article': article,
    }
    collection.insert_one(productdict)
    
    print(title)
    print(article)
    print('------------------------------------------------------')

NASA Wins Two Emmy Awards for Interactive Mission Coverage
NASA-JPL's coverage of the Mars InSight landing earns one of the two wins, making this the NASA center's second Emmy.
------------------------------------------------------
NASA's Mars 2020 Rover Completes Its First Drive
In a 10-plus-hour marathon, the rover steered, turned and drove in 3-foot (1-meter) increments over small ramps.
------------------------------------------------------
10.9 Million Names Now Aboard NASA's Perseverance Mars Rover
As part of NASA's 'Send Your Name to Mars' campaign, they've been stenciled onto three microchips along with essays from NASA's 'Name the Rover' contest. Next stop: Mars.
------------------------------------------------------
NASA Moves Forward With Campaign to Return Mars Samples to Earth
During this next phase, the program will mature critical technologies and make critical design decisions as well as assess industry partnerships.
-----------------------------------------------------

In [7]:
# New Url for Image pull
image_url = "https://spaceimages-mars.com/"
browser.visit(image_url)

In [8]:
# request the set url
time.sleep(1)
image_html = browser.html

In [9]:
image_soup=bs(image_html, 'html.parser')
print(image_soup.prettify)

<bound method Tag.prettify of <html class=""><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-

In [10]:
# Find and pulls featured first Image on the site
path = image_soup.find_all('img')[1]['src']
final_image= image_url + path
print(final_image)

https://spaceimages-mars.com/image/featured/mars1.jpg


In [11]:
# Url for Facts Table
table_url = "https://galaxyfacts-mars.com/"
browser.visit(table_url)

In [12]:
# Pulls both Tables
table = pd.read_html(table_url)
table

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [13]:
# Only pulls the comparison table
df=table[0]
df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
# Renames Columns
X=df.rename(columns={
    0: 'Mars-Earth Comparison',
    1: 'Mars',
    2: 'Earth'
})

In [15]:
# Drops first row
Y=X.drop([0])
Y

,Mars-Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [16]:
# Sets index
Y.set_index('Mars-Earth Comparison')

,Mars,Earth
Mars-Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [44]:
# Url for hemisphere image
hemispheres_url = "https://marshemispheres.com/"
browser.visit(hemispheres_url)

In [45]:
time.sleep(1)
hemispheres_html = browser.html

In [46]:
hemispheres_soup=bs(hemispheres_html, 'html.parser')
print(hemispheres_soup.prettify)

<bound method Tag.prettify of <html lang="en"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
<title>Astropedia Search Results | GUSS Astrogeology Science Center</title>
<meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<link href="css/main.css" media="screen" rel="stylesheet"/>
<link href="css/print.css" media="print" rel="stylesheet"/>
<link href="#" rel="icon" type="image/x-ico"/>
</head>
<body id="results">
<header>
<a href="#" style="float:right;margin-top:10px;" target="_blank">
<img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_2x.png"/>


In [47]:
Hemisresults = hemispheres_soup.find_all('div', class_='item')

In [56]:
#Storage space for info
hemisphereimage = []

#Loops
for h in Hemisresults:
        
    try:
        # Extract title
        hem=h.find('div',class_='description')
        X=hem.h3.text
        #finds image url
        pic = hem.a['href']
    
        #opens the image url
        browser.visit(hemispheres_url + pic)
        hemis_html = browser.html
        hemis_soup = bs(hemis_html, 'html.parser')
        hemis_link = hemis_soup.find('div', class_='downloads')
        hemis_url = hemis_link.find('li').a['href']
        
        #create Dictionary
        hemisdct = {}
        hemisdct['title'] = X
        hemisdct['hemis_url'] = hemispheres_url + hemis_url
        hemisphereimage.append(hemisdct)
        
        #print to check info
        print(hemisdct['title'])
        print(hemisdct['hemis_url'])
        
    except Exception as e:
        print(e)
    



Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/full.jpg
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg


In [25]:
#final dictionary
marsdct = {"Title": title, "Article": article,"Final_Image": final_image, "Comparison_Table": Y, "Hemispheres": hemisphereimage}
marsdct

{'Title': "NASA's Perseverance Rover Will Peer Beneath Mars' Surface ",
 'Article': "The agency's newest rover will use the first ground-penetrating radar instrument on the Martian surface to help search for signs of past microbial life. ",
 'Final_Image': 'https://spaceimages-mars.com/image/featured/mars1.jpg',
 'Comparison_Table':   Mars-Earth Comparison             Mars            Earth
 1             Diameter:         6,779 km        12,742 km
 2                 Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                Moons:                2                1
 4    Distance from Sun:   227,943,824 km   149,598,262 km
 5       Length of Year:   687 Earth days      365.24 days
 6          Temperature:     -87 to -5 °C      -88 to 58°C,
 'Hemispheres': ['https://marshemispheres.com/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png',
  'https://marshemispheres.com/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png',
  'https://marshemisphe

In [42]:
#quit browser
browser.quit()